## Risk and Return
**Return** is the **gain or loss** you make on an investment over time.

Return = ((Ending Value - Starting Value) / Starting Value) × 100%


**Types of Return**
- **Daily Return** – % change in price day to day
- **Annual Return** – Total return in one year
- **Cumulative Return** – Total return over multiple years
- **Dividend Return** – Income received from dividends

Example
You buy a stock for $100 and sell it for $120:
Return = ((120 - 100) / 100) × 100% = 20%



**Risk** is the **uncertainty or volatility** in an investment’s returns — how much its value may go up or down.

> More ups and downs = more risk.

**Types of Risk**
| Type                 | Description                                      |
|----------------------|--------------------------------------------------|
| **Market Risk**      | Price falls due to market-wide events            |
| **Interest Rate Risk** | Value drops when interest rates rise           |
| **Credit Risk**      | The company might default on debt                |
| **Liquidity Risk**   | Hard to sell the investment quickly              |
| **Volatility**       | How much the price fluctuates (a key measure)    |

**How is Risk measured**
- **Standard Deviation of Returns** – More variation = higher risk
- **Sharpe Ratio** – Return per unit of risk (higher is better)
- **Beta** – Volatility relative to the market (beta > 1 is riskier)


> The **higher the return**, the more **risk** you're usually taking on.

| Investment Type    | Expected Return | Risk Level     |
|--------------------|------------------|----------------|
| Savings account     | Low (1–3%)       | Very low       |
| Government bonds    | Low–moderate     | Low            |
| Blue-chip stocks    | Moderate (7–10%) | Moderate       |
| REITs               | Moderate         | Moderate       |
| Crypto/startups     | High (20%+)      | Very high      |



### Importing the required packages
- Yahoo Finance ( `yfinance` )
- Date time ( `datetime` )
- Plotly ( `plotly` )
- numpy ( `numpy` )

In [13]:
import datetime as dt
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

Next, we are going to define the starting date and ending date, for the ending date, it is going to be the current date, for the starting date, we are going to use `timeDelta` and choose a date 3 years from current date 

In [14]:
enddate = dt.datetime.now()
startdate = enddate - dt.timedelta(days=365*3)
stocks = ["AAPL","MSFT","AMZN","GOOGL","NVDA"]
df = yf.download(stocks,start=startdate,end=enddate,auto_adjust=True)
closeData = df['Close']

[*********************100%***********************]  5 of 5 completed


Here we are calculating the log of returns, for this we are going to use `numpy` and the `log` method defined in it. 

This  calculates the logarithmic returns of the `closeData` prices. It divides each day's closing price by the previous day's closing price, then takes the natural logarithm of that ratio. The `.dropna()` removes the first row which is NaN because there is no previous day’s price for the first data point. The result is a series or DataFrame of daily log returns.

In [15]:
logReturns = np.log(closeData / closeData.shift(1)).dropna()
dailyStdDev = logReturns.std()
annualizeStdDev = dailyStdDev * np.sqrt(252)
print(logReturns.head())
print(annualizeStdDev)

Ticker          AAPL      AMZN     GOOGL      MSFT      NVDA
Date                                                        
2022-07-06  0.009561  0.007286  0.011491  0.012701  0.011032
2022-07-07  0.023716  0.017342  0.036095  0.008193  0.046994
2022-07-08  0.004704 -0.006814  0.004791 -0.002761 -0.001262
2022-07-11 -0.014868 -0.033353 -0.031292 -0.011839 -0.044280
2022-07-12  0.006811 -0.022900 -0.014419 -0.041845 -0.004631
Ticker
AAPL     0.280670
AMZN     0.349291
GOOGL    0.322567
MSFT     0.263184
NVDA     0.535275
dtype: float64


In [16]:
rows = 3
cols = 2
fig_histogram = make_subplots(rows=rows, cols=cols)
traces = [
    go.Histogram(x=logReturns[stock], name=stock) for stock in stocks
]

for idx, trace in enumerate(traces):
    row, col = divmod(idx, cols)
    fig_histogram.add_trace(trace, row=row+1, col=col+1)

fig_histogram.update_layout(autosize=False, width=1000, height=800, title="Frequency of Log Returns")
for idx, stock in enumerate(stocks):
    row, col = divmod(idx, cols)
    fig_histogram.update_xaxes(
        title_text=f'{stock} Annualize Volatility: {np.round(annualizeStdDev[stock]*100, 1)}%',
        row=row+1,
        col=col+1
    )

fig_histogram.show()